# IPO 기업목록에 대응하는 투자설명서목록 추출

라이브러리 및 파일 불러오기

In [168]:
import pandas as pd
import numpy as np
import requests
import json

import matplotlib.pyplot as plt
import datetime  # "문자열" 날짜를 "날짜형식" 날짜로 표현함
from datetime import datetime, timedelta

In [25]:
############################################################################################ 
##### KIND data (IPO정보) + DATAGUIDE data (상장일) => Sample firm list 제작
############################################################################################ 

path = 'D:\\cg\\esopdata\\rawdata\\'
path1 = 'D:\\cg\\esopdata\\data\\'
list_de_tr = pd.read_excel(path+'basic_data_raw.xlsx',sheetname='list_de_tr')
kitype = pd.read_csv(path+'kitype_raw.csv',engine='python')

In [135]:
print(list_de_tr.shape, kitype.shape)

(8616, 54) (2299, 13)


In [130]:
############################################################################################ 
##### 1. 분석기간 동안의 상장사 기업정보 추출 (DATAGUIDE): firm-level 2,872
##### 1-1. 상장일만 추출
############################################################################################
listdate = list_de_tr.set_index(['Symbol','Item Name'])
listdate = listdate.loc[(slice(None),'상장(등록)일자'),:]
listdate = listdate.reset_index().drop(columns='Item Name').T.drop_duplicates().reset_index(drop=True).T
listdate = listdate.rename(columns={0:'Symbol',1:'listdate'})
listdate.set_index('Symbol',inplace=True)

############################################################################################ 
##### 1-2. 상장일이 분석기간 내에 있는 기업들만 추출: firm-level 821 (2008.12.31-2018.12.31)
############################################################################################ 
listdate.listdate = pd.to_datetime(listdate.listdate, format ='%Y%m%d')
listdate = listdate[ ( listdate.listdate>='2008-12-31') & (listdate.listdate<='2018-12-31') ]

print('결과: Total IPO firms',len(listdate),'(DATAGUIDE)')
listdate.head()

결과: Total IPO firms 821 (DATAGUIDE)


listdate
Symbol            
A207940 2016-11-10
A028260 2014-12-18
A018260 2014-11-14
A034730 2009-11-11
A032830 2010-05-12

In [145]:
kitype[kitype.type.str.contains('재상장')]

Symbol Symbol Name    listdate    type security  \
0     A319660       피에스케이  2019-05-10     재상장       주권   
77    A306200        세아제강  2018-10-05     재상장       주권   
101   A300720       한일시멘트  2018-08-06     재상장       주권   
110   A298000        효성화학  2018-07-13     재상장       주권   
111   A298020       효성티앤씨  2018-07-13     재상장       주권   
112   A298040       효성중공업  2018-07-13     재상장       주권   
113   A298050      효성첨단소재  2018-07-13     재상장       주권   
121   A294870   HDC현대산업개발  2018-06-12     재상장       주권   
150   A284740       쿠쿠홈시스  2018-01-11     재상장       주권   
151   A285130       SK케미칼  2018-01-05     재상장       주권   
163   A282330      BGF리테일  2017-12-08     재상장       주권   
167   A281820        케이씨텍  2017-12-05     재상장       주권   
172   A282690       동아타이어  2017-11-29     재상장       주권   
182   A280360        롯데제과  2017-10-30     재상장       주권   
219   A271980        제일약품  2017-07-17     재상장       주권   
223   A272290     이녹스첨단소재  2017-07-10     재상장       주권   
225   A271560         오리온  2017-07-07     재상장       주권   
237   A267980        매일유업  2017-06-05     재상장       주권   
243   A268280       미원에스씨  2017-05-22     재상장       주권   
245   A267290      경동도시가스  2017-05-12     재상장       주권   
247   A267250      현대로보틱스  2017-05-10     재상장       주권   
248   A267260      현대일렉트릭  2017-05-10     재상장       주권   
249   A267270      현대건설기계  2017-05-10     재상장       주권   
259   A264900       크라운제과  2017-04-11     재상장       주권   
260   A265520       AP시스템  2017-04-07     재상장       주권   
263   A264450        유비쿼스  2017-03-31     재상장       주권   
344   A249420        일동제약  2016-08-31     재상장       주권   
348   A248170        샘표식품  2016-08-09     재상장       주권   
381   A243070         휴온스  2016-06-03     재상장       주권   
390   A240810       원익IPS  2016-05-02     재상장       주권   
...       ...         ...         ...     ...      ...   
1620  A071090        하이스틸  2003-02-17     재상장       주권   
1660  A070080       그로웰메탈  2002-12-13     재상장       주권   
1664  A069960       현대백화점  2002-11-25     재상장       주권   
1668  A069460       대호에이엘  2002-11-11     재상장       주권   
1669  A069470       남선홈웨어  2002-11-11     재상장       주권   
1675  A069620        대웅제약  2002-11-01     재상장       주권   
1681  A069260         휴켐스  2002-10-07     재상장       주권   
1696  A068870      LG생명과학  2002-08-16     재상장       주권   
1700  A068290       삼성출판사  2002-08-05     재상장       주권   
1705  A067830         유레스  2002-07-25     재상장       주권   
1725  A064420      케이피케미칼  2002-06-21     재상장       주권   
1731  A067130        룸앤데코  2002-06-17     재상장       주권   
1744  A066330     시큐리티코리아  2002-05-17     재상장       주권   
1760  A066570        LG전자  2002-04-22     재상장       주권   
1769  A064960        대우정밀  2002-03-11     재상장       주권   
1842  A063510    코오롱인터내셔널  2001-12-24     재상장       주권   
1862  A063160      종근당바이오  2001-12-11     재상장       주권   
1881  A060000        국민은행  2001-11-09  재상장 합병       주권   
1911  A058820        피엠케이  2001-08-31     재상장       주권   
1944  A058650       세아홀딩스  2001-07-30     재상장       주권   
1988  A051900      LG생활건강  2001-04-25     재상장       주권   
1989  A051910        LG화학  2001-04-25     재상장       주권   
1995  A047040        대우건설  2001-03-23     재상장       주권   
1996  A047050     대우인터내셔널  2001-03-23     재상장       주권   
2001  A042660      대우조선공업  2001-02-02     재상장       주권   
2002  A042670      대우종합기계  2001-02-02     재상장       주권   
2005  A051630        진양화학  2001-01-29     재상장       주권   
2051  A049770       동원F&B  2000-11-23     재상장       주권   
2073  A047060        풍성전기  2000-09-08     재상장       주권   
2289  A036580        대상사료  1999-10-25     재상장       주권   

                           business sectors nationality        underwriter  \
0                             특수 목적용 기계 제조업        대한민국            키움증권(주)   
77                                1차 철강 제조업        대한민국       엔에이치투자증권주식회사   
101                시멘트, 석회, 플라스터 및 그 제품 제조업        대한민국            삼성증권(주)   
110                             기초 화학물질 제조업        대한민국  대신증권(주),미래에셋대우(주)   
111                 

In [151]:
############################################################################################ 
##### 2. 분석기간 동안의 IPO상장 정보 추출 (KIND): IPO기업수 firm-level 2,299
##### 2-1. IPO상장 Data Cleansing
############################################################################################ 
kind_type = kitype.drop(['business sectors','액면가(원)','주요제품'],axis=1)
kind_type.rename(columns={'공모가(원)':'offer_price','공모금액(천원)':'proceeds(1000)','최초상장주식수(주)':'initial_shares'},inplace=True)
kind_type.set_index('Symbol',inplace=True)

############################################################################################ 
##### 2-2. 상장일이 분석기간 내에 있는 기업들만 추출: firm-level 1,102 (2008.12.31-2018.12.31)
############################################################################################ 
kind_type.listdate = pd.to_datetime(kind_type.listdate, format ='%Y-%m-%d')
kind_type = kind_type[ ( kind_type.listdate>='2008-12-31') & (kind_type.listdate<='2018-12-31') ]


############################################################################################ 
##### 2-3. 제외대상 반영 (SPAC, 재상장, 이전상장 / REITS, 선박투자회사, Infra Fund, Invest. Company, ETF, ETN
############################################################################################ 

####### 재상장
def remove_relist(type):
    if "재상장" in type:
        return False
    else:
        return True

####### 이전상장
def remove_transfer(type):
    if "이전상장" in type:
        return False
    else:
        return True

####### SPAC
def remove_spac(company):
    if "스팩" in company:
        return False
    else: 
        return True
    
############################################################################################ 
print('01.재상장: ', len(kind_type[kind_type.type.str.contains('재상장')]))
print('02.이전상장 ',len(kind_type[kind_type.type.str.contains('이전상장')]))
print('03.SPAC ',    len(kind_type[kind_type['Symbol Name'].str.contains('스팩')]))

kind_type = kind_type [kind_type.type.apply(remove_relist)]  
kind_type = kind_type [kind_type.type.apply(remove_transfer)]
kind_type = kind_type [kind_type['Symbol Name'].apply(remove_spac)]  
############################################################################################ 

####### 외국주권
def remove_etc1(security):
    if "외국주권" in security:
        return False
    else:
        return True

####### 주식예탁증권
def remove_etc2(security):
    if "주식예탁증권" in security:
        return False
    else:
        return True

####### REITS
def remove_etc3(security):
    if "부동산투자회사" in security:
        return False
    else:
        return True

####### 선박투자회사
def remove_etc4(security):
    if "선박투자회사" in security:
        return False
    else:
        return True

####### 투자회사
def remove_etc5(security):
    if "투자회사" in security:
        return False
    else:
        return True
    
############################################################################################ 
print('04.외국주권: ',    len(kind_type[kind_type.security.str.contains('외국주권')]))
print('05.주식예탁증권: ',len(kind_type[kind_type.security.str.contains('주식예탁증권')]))
print('06.REITS ',        len(kind_type[kind_type.security.str.contains('부동산투자회사')]))
print('07.선박투자회사 ', len(kind_type[kind_type.security.str.contains('선박투자회사')]))
print('08.투자회사 ',     len(kind_type[kind_type.security.str.contains('투자회사')]))

kind_type = kind_type [kind_type['security'].apply(remove_etc1)]  
kind_type = kind_type [kind_type['security'].apply(remove_etc2)]
kind_type = kind_type [kind_type['security'].apply(remove_etc2)]
kind_type = kind_type [kind_type['security'].apply(remove_etc2)]
kind_type = kind_type [kind_type['security'].apply(remove_etc2)]
############################################################################################ 

print('결과: Total IPO firms',len(kind_type),'(KIND)')
kind_type.head()

01.재상장:  84
02.이전상장  45
03.SPAC  144
04.외국주권:  21
05.주식예탁증권:  8
06.REITS  11
07.선박투자회사  9
08.투자회사  22
결과: Total IPO firms 800 (KIND)


Symbol Name   listdate  type security nationality  \
Symbol                                                      
A311060        엘에이티 2018-12-28  신규상장       주권        대한민국   
A298690        에어부산 2018-12-27  신규상장       주권        대한민국   
A311840        대원모방 2018-12-27  신규상장       주권        대한민국   
A238200         비피도 2018-12-26  신규상장       주권        대한민국   
A314130      지놈앤컴퍼니 2018-12-26  신규상장       주권        대한민국   

                    underwriter offer_price proceeds(1000) initial_shares  
Symbol                                                                     
A311060                 KB증권(주)           -              -      1,059,100  
A298690  미래에셋대우(주),엔에이치투자증권주식회사       3,600     18,745,200     52,070,000  
A311840                 교보증권(주)           -              -      1,593,280  
A238200               한국투자증권(주)      18,000      7,380,018      4,090,000  
A314130               한국투자증권(주)           -              -      7,703,997

In [188]:
############################################################################################ 
##### 3. DATA MERGE를 통한 기업목록 생성: 상장사 기업정보 (DATAGUIDE) + IPO상장 정보 (KIND)
##### 3-1. DATA MERGE : KONEX, 인적분할 상장 등 제외함
############################################################################################ 
company_list = pd.merge(listdate,kind_type,on='Symbol')
company_list = company_list.rename(columns={'listdate_x':'listdate'}).drop(['nationality','type','security','listdate_y'],axis=1)
company_list.reset_index(inplace=True)
print('결과: Total IPO firms',len(company_list),'(merged)')
company_list.head()
# 과거 d2 대신에 company_list 쓰기로 하자.

결과: Total IPO firms 575 (merged)


Symbol   listdate Symbol Name                            underwriter  \
0  A207940 2016-11-10    삼성바이오로직스               한국투자증권(주),씨티그룹글로벌마켓증권(주)   
1  A028260 2014-12-18        제일모직                              미래에셋대우(주)   
2  A018260 2014-11-14     삼성에스디에스  한국투자증권(주),골드만삭스증권 서울지점,제이피모간증권회사 서울지점   
3  A034730 2009-11-11      SK C&C         한국투자증권(주),엔에이치투자증권주식회사,대신증권(주)   
4  A032830 2010-05-12        삼성생명                 한국투자증권(주),골드만삭스증권 서울지점   

  offer_price proceeds(1000) initial_shares  
0     136,000  2,249,617,072     66,165,000  
1      53,000  1,523,747,350    135,000,000  
2     190,000  1,158,924,760     77,377,800  
3      30,000    540,000,000     50,000,000  
4     110,000  4,888,116,200    200,000,000

In [189]:
company_list.to_csv(path1+'company_list.csv',encoding='ms949', header=True,index=False)

In [184]:
############################################################################################ 
##### 4. 각 기업의 IPO 당시 투자설명서 목록 추출
##### 4-1. DATA MERGE : KONEX, 인적분할 상장 등 제외함
############################################################################################ 

####### 문서제목에 '투자설명서'을 포함할 것
def prospectus(filename):
    if "투자설명서" in filename:
        return True
    else: 
        return False

In [ ]:
##### (1) 기업목록에서 ~100번까지 추출: 103 #####################################
report = pd.DataFrame()

for i in range(0,100):
    #############################################################################
    apikey = "ccc77322884d812b24884f9ed2789cb1e6c4b71f" # API 본인인증 번호 입력
    crpcode = company_list.index.tolist()[i]
    startdate = "20081231"
    enddate = "20181231"
    finalver = "Y"
    typedis = "C"    # 발행공시 목적의 보고서 only: '투자설명서'가 있는 영역임
    #############################################################################
    url = "http://dart.fss.or.kr/api/search.json?auth={0}&crp_cd={1}&start_dt={2}&end_dt{3}&fin_rpt={4}&dsp_tp={5}&page_set=100"
    url = url.format(apikey, crpcode, startdate, enddate, finalver, typedis)
    
    #############################################################################
    response = requests.get(url)
    data = json.loads(response.content)
    cd = pd.DataFrame(data['list'])

    if not cd.empty:  # DataFrame가 공백이 아니라면, 아래를 실행하라.
        rcd = cd[cd['rpt_nm'].apply(prospectus)]
        report = pd.concat([report, rcd]) # total_re = total_re.append(rcd)와 같은 결과임
        report
report = report.drop_duplicates()
len(report)

In [ ]:
##### (2) 기업목록에서 ~200번까지 추출: 231 (+128)###############################
for i in range(100,200):
    #############################################################################
    apikey = "ccc77322884d812b24884f9ed2789cb1e6c4b71f" # API 본인인증 번호 입력
    crpcode = company_list.index.tolist()[i]
    startdate = "20081231"
    enddate = "20181231"
    finalver = "Y"
    typedis = "C"    # 발행공시 목적의 보고서 only: '투자설명서'가 있는 영역임
    #############################################################################
    url = "http://dart.fss.or.kr/api/search.json?auth={0}&crp_cd={1}&start_dt={2}&end_dt{3}&fin_rpt={4}&dsp_tp={5}&page_set=100"
    url = url.format(apikey, crpcode, startdate, enddate, finalver, typedis)
    
    #############################################################################
    response = requests.get(url)
    data = json.loads(response.content)
    cd = pd.DataFrame(data['list'])

    if not cd.empty:
        rcd = cd[cd['rpt_nm'].apply(prospectus)]
        report = pd.concat([report, rcd])
        report
report = report.drop_duplicates()
len(report)

In [ ]:
##### (3) 기업목록에서 ~300번까지 추출: 382 (+151)###############################
for i in range(200,300):
    #############################################################################
    apikey = "ccc77322884d812b24884f9ed2789cb1e6c4b71f" # API 본인인증 번호 입력
    crpcode = company_list.index.tolist()[i]
    startdate = "20081231"
    enddate = "20181231"
    finalver = "Y"
    typedis = "C"    # 발행공시 목적의 보고서 only: '투자설명서'가 있는 영역임
    #############################################################################
    url = "http://dart.fss.or.kr/api/search.json?auth={0}&crp_cd={1}&start_dt={2}&end_dt{3}&fin_rpt={4}&dsp_tp={5}&page_set=100"
    url = url.format(apikey, crpcode, startdate, enddate, finalver, typedis)
    
    #############################################################################
    response = requests.get(url)
    data = json.loads(response.content)
    cd = pd.DataFrame(data['list'])

    if not cd.empty:
        rcd = cd[cd['rpt_nm'].apply(prospectus)]
        report = pd.concat([report, rcd])
        report
report = report.drop_duplicates()
len(report)

In [ ]:
##### (4) 기업목록에서 ~400번까지 추출: 591 (+209)###############################
for i in range(300,400):
    #############################################################################
    apikey = "ccc77322884d812b24884f9ed2789cb1e6c4b71f" # API 본인인증 번호 입력
    crpcode = company_list.index.tolist()[i]
    startdate = "20081231"
    enddate = "20181231"
    finalver = "Y"
    typedis = "C"    # 발행공시 목적의 보고서 only: '투자설명서'가 있는 영역임
    #############################################################################
    url = "http://dart.fss.or.kr/api/search.json?auth={0}&crp_cd={1}&start_dt={2}&end_dt{3}&fin_rpt={4}&dsp_tp={5}&page_set=100"
    url = url.format(apikey, crpcode, startdate, enddate, finalver, typedis)
    
    #############################################################################
    response = requests.get(url)
    data = json.loads(response.content)
    cd = pd.DataFrame(data['list'])

    if not cd.empty:
        rcd = cd[cd['rpt_nm'].apply(prospectus)]
        report = pd.concat([report, rcd])
        report
report = report.drop_duplicates()
len(report)

In [ ]:
##### (5) 기업목록에서 ~500번까지 추출 : 820 (+229)##############################
for i in range(400,500):
    #############################################################################
    apikey = "ccc77322884d812b24884f9ed2789cb1e6c4b71f" # API 본인인증 번호 입력
    crpcode = company_list.index.tolist()[i]
    startdate = "20081231"
    enddate = "20181231"
    finalver = "Y"
    typedis = "C"    # 발행공시 목적의 보고서 only: '투자설명서'가 있는 영역임
    #############################################################################
    url = "http://dart.fss.or.kr/api/search.json?auth={0}&crp_cd={1}&start_dt={2}&end_dt{3}&fin_rpt={4}&dsp_tp={5}&page_set=100"
    url = url.format(apikey, crpcode, startdate, enddate, finalver, typedis)
    
    #############################################################################
    response = requests.get(url)
    data = json.loads(response.content)
    cd = pd.DataFrame(data['list'])

    if not cd.empty:
        rcd = cd[cd['rpt_nm'].apply(prospectus)]
        report = pd.concat([report, rcd])
        report
report = report.drop_duplicates()
len(report)

In [23]:
##### (6) 기업목록에서 ~끝까지 추출: 1045 (+225)################################
for i in range(400,len(company_list)):
    #############################################################################
    apikey = "ccc77322884d812b24884f9ed2789cb1e6c4b71f" # API 본인인증 번호 입력
    crpcode = company_list.index.tolist()[i]
    startdate = "20081231"
    enddate = "20181231"
    finalver = "Y"
    typedis = "C"    # 발행공시 목적의 보고서 only: '투자설명서'가 있는 영역임
    #############################################################################
    url = "http://dart.fss.or.kr/api/search.json?auth={0}&crp_cd={1}&start_dt={2}&end_dt{3}&fin_rpt={4}&dsp_tp={5}&page_set=100"
    url = url.format(apikey, crpcode, startdate, enddate, finalver, typedis)
    
    #############################################################################
    response = requests.get(url)
    data = json.loads(response.content)
    cd = pd.DataFrame(data['list'])

    if not cd.empty:
        rcd = cd[cd['rpt_nm'].apply(prospectus)]
        report = pd.concat([report, rcd])
        report
report = report.drop_duplicates()
len(report)

http://dart.fss.or.kr/api/search.json?auth=28c30ebb6f7dbf43aec08b93f8f4cb996c831c60&crp_cd=100030&start_dt=20081231&end_dt20181231&fin_rpt=Y&dsp_tp=C&page_set=100
   crp_cd crp_cls crp_nm flr_nm    rcp_dt          rcp_no rmk  \
0  100030       K  모바일리더  모바일리더  20100519  20100519000063       
1  100030       K  모바일리더  모바일리더  20100511  20100511000812       
2  100030       K  모바일리더  모바일리더  20100511  20100511000804       

                rpt_nm  
0            증권발행실적보고서  
1          [기재정정]투자설명서  
2  [발행조건확정]증권신고서(지분증권)  
http://dart.fss.or.kr/api/search.json?auth=28c30ebb6f7dbf43aec08b93f8f4cb996c831c60&crp_cd=060980&start_dt=20081231&end_dt20181231&fin_rpt=Y&dsp_tp=C&page_set=100
    crp_cd crp_cls crp_nm flr_nm    rcp_dt          rcp_no rmk  \
0   060980       Y  한라홀딩스  한라홀딩스  20190923  20190923001458       
1   060980       Y  한라홀딩스  한라홀딩스  20180314  20180314000511       
2   060980       Y  한라홀딩스  한라홀딩스  20180314  20180314000010       
3   060980       Y  한라홀딩스  한라홀딩스  20180313  20180

2        [기재정정]투자설명서  
http://dart.fss.or.kr/api/search.json?auth=28c30ebb6f7dbf43aec08b93f8f4cb996c831c60&crp_cd=106080&start_dt=20081231&end_dt20181231&fin_rpt=Y&dsp_tp=C&page_set=100
   crp_cd crp_cls  crp_nm  flr_nm    rcp_dt          rcp_no rmk  \
0  106080       K  지투하이소닉  지투하이소닉  20160519  20160519000133       
1  106080       K  지투하이소닉  지투하이소닉  20160512  20160512000039       
2  106080       K  지투하이소닉  지투하이소닉  20160511  20160511003278       
3  106080       K  지투하이소닉  지투하이소닉  20100203  20100203000059       
4  106080       K  지투하이소닉  지투하이소닉  20100126  20100126000046       
5  106080       K  지투하이소닉  지투하이소닉  20100126  20100126000041       

                rpt_nm  
0            증권발행실적보고서  
1                투자설명서  
2  [발행조건확정]증권신고서(채무증권)  
3            증권발행실적보고서  
4          [기재정정]투자설명서  
5  [발행조건확정]증권신고서(지분증권)  
http://dart.fss.or.kr/api/search.json?auth=28c30ebb6f7dbf43aec08b93f8f4cb996c831c60&crp_cd=101330&start_dt=20081231&end_dt20181231&fin_rpt=Y&dsp_tp=C&page_set=100
   crp

   crp_cd crp_cls crp_nm flr_nm    rcp_dt          rcp_no rmk  \
0  115390       Y    락앤락    락앤락  20111103  20111103000189       
1  115390       Y    락앤락    락앤락  20111020  20111020000196       
2  115390       Y    락앤락    락앤락  20111020  20111020000193       
3  115390       Y    락앤락    락앤락  20100126  20100126000017       
4  115390       Y    락앤락    락앤락  20100118  20100118000248       
5  115390       Y    락앤락    락앤락  20100118  20100118000247       

                rpt_nm  
0            증권발행실적보고서  
1          [기재정정]투자설명서  
2  [발행조건확정]증권신고서(지분증권)  
3            증권발행실적보고서  
4          [기재정정]투자설명서  
5  [발행조건확정]증권신고서(지분증권)  
http://dart.fss.or.kr/api/search.json?auth=28c30ebb6f7dbf43aec08b93f8f4cb996c831c60&crp_cd=115440&start_dt=20081231&end_dt20181231&fin_rpt=Y&dsp_tp=C&page_set=100
   crp_cd crp_cls crp_nm flr_nm    rcp_dt          rcp_no rmk  \
0  115440       K    우리넷    우리넷  20100125  20100125000044       
1  115440       K    우리넷    우리넷  20100114  20100114000233       
2  115440  

   crp_cd crp_cls crp_nm flr_nm    rcp_dt          rcp_no rmk  \
0  096640       K    멜파스    멜파스  20181213  20181213000270       
1  096640       K    멜파스    멜파스  20181206  20181206000050       
2  096640       K    멜파스    멜파스  20181206  20181206000048       
3  096640       K    멜파스    멜파스  20091218  20091218000111       
4  096640       K    멜파스    멜파스  20091204  20091204000096       
5  096640       K    멜파스    멜파스  20091204  20091204000094       

                rpt_nm  
0            증권발행실적보고서  
1                투자설명서  
2  [발행조건확정]증권신고서(채무증권)  
3            증권발행실적보고서  
4  [발행조건확정]증권신고서(지분증권)  
5          [기재정정]투자설명서  
http://dart.fss.or.kr/api/search.json?auth=28c30ebb6f7dbf43aec08b93f8f4cb996c831c60&crp_cd=114410&start_dt=20081231&end_dt20181231&fin_rpt=Y&dsp_tp=C&page_set=100
     crp_cd crp_cls   crp_nm   flr_nm    rcp_dt          rcp_no rmk  \
0  00359641       E  현대푸드시스템  현대푸드시스템  20091215  20091215000053       
1  00359641       E  현대푸드시스템  현대푸드시스템  20091203  20091203000135 

   crp_cd crp_cls    crp_nm    flr_nm    rcp_dt          rcp_no rmk  \
0  101000       K  상상인인더스트리  상상인인더스트리  20180327  20180327000871       
1  101000       K  상상인인더스트리  상상인인더스트리  20180326  20180326000453       
2  101000       K  상상인인더스트리  상상인인더스트리  20170313  20170313000454       
3  101000       K  상상인인더스트리  상상인인더스트리  20170309  20170309000763       
4  101000       K  상상인인더스트리  상상인인더스트리  20131224  20131224000049       
5  101000       K  상상인인더스트리  상상인인더스트리  20131212  20131212000028       
6  101000       K  상상인인더스트리  상상인인더스트리  20131212  20131212000026       
7  101000       K  상상인인더스트리  상상인인더스트리  20091103  20091103000186       
8  101000       K  상상인인더스트리  상상인인더스트리  20091023  20091023000159       
9  101000       K  상상인인더스트리  상상인인더스트리  20091023  20091023000152       

                 rpt_nm  
0             소액공모실적보고서  
1  [기재정정]소액공모공시서류(지분증권)  
2             소액공모실적보고서  
3        소액공모공시서류(지분증권)  
4             증권발행실적보고서  
5           [기재정정]투자설명서  
6   [발행조건확정]증권신고서(지분증권)  
7         

   crp_cd crp_cls crp_nm flr_nm    rcp_dt          rcp_no rmk  \
0  109740       K  디에스케이  디에스케이  20090921  20090921000057       
1  109740       K  디에스케이  디에스케이  20090910  20090910000155       
2  109740       K  디에스케이  디에스케이  20090910  20090910000153       

                rpt_nm  
0            증권발행실적보고서  
1  [발행조건확정]증권신고서(지분증권)  
2          [기재정정]투자설명서  
http://dart.fss.or.kr/api/search.json?auth=28c30ebb6f7dbf43aec08b93f8f4cb996c831c60&crp_cd=095700&start_dt=20081231&end_dt20181231&fin_rpt=Y&dsp_tp=C&page_set=100
   crp_cd crp_cls crp_nm flr_nm    rcp_dt          rcp_no rmk  \
0  095700       K    제넥신    제넥신  20190820  20190820000005       
1  095700       K    제넥신    제넥신  20190715  20190715000014   철   
2  095700       K    제넥신    제넥신  20190703  20190703000465   철   
3  095700       K    제넥신    제넥신  20090914  20090914000068       
4  095700       K    제넥신    제넥신  20090904  20090904000242       
5  095700       K    제넥신    제넥신  20090904  20090904000194       

                rpt_

   crp_cd crp_cls crp_nm flr_nm    rcp_dt          rcp_no rmk  \
0  053690       Y  한미글로벌  한미글로벌  20090617  20090617000095       
1  053690       Y  한미글로벌  한미글로벌  20090605  20090605000521       
2  053690       Y  한미글로벌  한미글로벌  20090605  20090605000516       

                       rpt_nm  
0               증권발행실적보고서[주식]  
1             [기재정정]투자설명서[주식]  
2  [기재정정]증권신고서[주식(모집설립제외)의경우]  
http://dart.fss.or.kr/api/search.json?auth=28c30ebb6f7dbf43aec08b93f8f4cb996c831c60&crp_cd=028100&start_dt=20081231&end_dt20181231&fin_rpt=Y&dsp_tp=C&page_set=100
   crp_cd crp_cls crp_nm flr_nm    rcp_dt          rcp_no rmk  \
0  028100       Y   동아지질   동아지질  20090610  20090609000402       
1  028100       Y   동아지질   동아지질  20090528  20090528000223       
2  028100       Y   동아지질   동아지질  20090528  20090528000220       

                       rpt_nm  
0               증권발행실적보고서[주식]  
1             [기재정정]투자설명서[주식]  
2  [기재정정]증권신고서[주식(모집설립제외)의경우]  
http://dart.fss.or.kr/api/search.json?auth=28c30ebb6f7dbf43

   crp_cd crp_cls crp_nm flr_nm    rcp_dt          rcp_no rmk  \
0  081150       K   티플랙스   티플랙스  20091124  20091124000056       
1  081150       K   티플랙스   티플랙스  20091106  20091106000034       
2  081150       K   티플랙스   티플랙스  20091106  20091106000032       
3  081150       K   티플랙스   티플랙스  20090421  20090421000103       
4  081150       K   티플랙스   티플랙스  20090410  20090410001825       
5  081150       K   티플랙스   티플랙스  20090410  20090410001798       

                       rpt_nm  
0                   증권발행실적보고서  
1         [발행조건확정]증권신고서(지분증권)  
2                 [기재정정]투자설명서  
3               증권발행실적보고서[주식]  
4             [기재정정]투자설명서[주식]  
5  [기재정정]증권신고서[주식(모집설립제외)의경우]  
http://dart.fss.or.kr/api/search.json?auth=28c30ebb6f7dbf43aec08b93f8f4cb996c831c60&crp_cd=071670&start_dt=20081231&end_dt20181231&fin_rpt=Y&dsp_tp=C&page_set=100
   crp_cd crp_cls   crp_nm   flr_nm    rcp_dt          rcp_no rmk  \
0  071670       K  에이테크솔루션  에이테크솔루션  20090415  20090414001680       
1  071670       K  

4             [기재정정]투자설명서[주식]  
http://dart.fss.or.kr/api/search.json?auth=28c30ebb6f7dbf43aec08b93f8f4cb996c831c60&crp_cd=086900&start_dt=20081231&end_dt20181231&fin_rpt=Y&dsp_tp=C&page_set=100
   crp_cd crp_cls crp_nm flr_nm    rcp_dt          rcp_no rmk         rpt_nm
0  086900       K   메디톡스   메디톡스  20090113  20090113000022      증권발행실적보고서[주식]


In [26]:
report.to_csv("D:\\cg\\data\\report.csv", encoding="ms949", header=True,index=False)
# Raw data: "report" csv파일, 보고서 갯수: 총 1045개

1059


crp_cd crp_cls     crp_nm     flr_nm    rcp_dt          rcp_no rmk  \
1  298690       Y       에어부산       에어부산  20181217  20181217000216       
1  238200       K        비피도        비피도  20181214  20181214000429       
2  263050       K       유틸렉스       유틸렉스  20181212  20181212000279       
1  290550       K       디케이티       디케이티  20181211  20181211000124       
1  299900       K    위지윅스튜디오    위지윅스튜디오  20181211  20181211000331       
1  298380       K    에이비엘바이오    에이비엘바이오  20181210  20181210000345       
1  100590       K        머큐리        머큐리  20181204  20181204000072       
1  110020       K     전진바이오팜     전진바이오팜  20181203  20181203000108       
1  270870       K        뉴트리        뉴트리  20181130  20181130000026       
1  302430       K      이노메트리      이노메트리  20181130  20181130000313       
1  307870       K  상상인이안1호스팩  상상인이안1호스팩  20181127  20181127000010       
1  290120       K      대유에이피      대유에이피  20181126  20181126000391       
1  128540       K        에코캡        에코캡  20181115  20181115000119       
1  299910       K        베스파        베스파  20181120  20181120000004       
1  111710       K       남화산업       남화산업  20181113  20181113000246       
2  179290       K       엠아이텍       엠아이텍  20181116  20181116000018       
1  117730       K      티로보틱스      티로보틱스  20181116  20181116000182       
1  246710       K    티앤알바이오팹    티앤알바이오팹  20181114  20181114000742       
1  290660       K       네오펙트       네오펙트  20181115  20181115000032       
1  267850       Y    아시아나IDT    아시아나IDT  20181114  20181114000972       
1  263690       K        디알젬        디알젬  20181108  20181108000221       
1  217330       K       싸이토젠       싸이토젠  20181112  20181112000316       
1  208340       K        파멥신        파멥신  20181108  20181108000408       
1  027360       K     아주IB투자     아주IB투자  20181109  20181109000388       
4  027360       K     아주IB투자     아주IB투자  20100629  20100629000023       
1  263020       K      디케이앤디      디케이앤디  20181106  20181106000213       
1  227100       K        디자인        디자인  20181107  20181107000089       
1  194700       K       노바렉스       노바렉스  20181102  20181102000071       
1  268600       K       셀리버리       셀리버리  20181025  20181025000031       
1  290670       K     대보마그네틱     대보마그네틱  20181023  20181023000171       

        rpt_nm  
1  [기재정정]투자설명서  
1  [기재정정]투자설명서  
2  [기재정정]투자설명서  
1  [기재정정]투자설명서  
1  [기재정정]투자설명서  
1  [기재정정]투자설명서  
1  [기재정정]투자설명서  
1  [기재정정]투자설명서  
1  [기재정정]투자설명서  
1  [기재정정]투자설명서  
1  [기재정정]투자설명서  
1  [기재정정]투자설명서  
1  [기재정정]투자설명서  
1  [기재정정]투자설명서  
1  [기재정정]투자설명서  
2  [기재정정]투자설명서  
1  [기재정정]투자설명서  
1  [기재정정]투자설명서  
1  [기재정정]투자설명서  
1  [기재정정]투자설명서  
1  [기재정정]투자설명서  
1  [기재정정]투자설명서  
1  [기재정정]투자설명서  
1  [기재정정]투자설명서  
4        투자설명서  
1  [기재정정]투자설명서  
1  [기재정정]투자설명서  
1  [기재정정]투자설명서  
1  [기재정정]투자설명서  
1  [기재정정]투자설명서

### '투자설명서' 목록 테이블 정리

In [58]:
############################################################################################ 
##### 5. 다중투자설명서 목록에서 비관련 제외
##### 5-1. 상장 비관련 투자설명서 제외
############################################################################################ 

## 문서명칭에 '철' 있으면 제거
report1 = report[report.rmk !='철'].reset_index(drop=True).drop(['rmk','crp_cls','flr_nm'],axis=1)
print(report.shape,report1.shape)

############################################################################################ 
##### 5-2. '투자설명서 공시일'과 '상장일' 간의 일수 차이 계산
############################################################################################ 

## 문서공시일 칼럼 생성
report1['redate'] = pd.to_datetime( report1['rcp_dt'].str[:8] )

## 기업코드 칼럼 정리
report1.rename(columns={'crp_cd':'Symbol'},inplace=True)
report1.dtypes
red2 = pd.merge(report1, d2, on ='Symbol')
red2['list_date'] = pd.to_datetime( red2['list_date'] )
red2["days"] = ( red2["list_date"] - red2["redate"] ).dt.days
print(red2["days"].head())

############################################################################################ 
##### 5-3. '투자설명서 공시일'과 '상장일' 간의 일수 차이 계산 및 추출
############################################################################################ 
## 투자설명서 공시일이 상장일 이전 대비 30일 초과시, OR 상장일 이후인 경우 ▶ 제외할 것
red2 = red2[(red2["days"] < 30)&(0<red2["days"])]
red2.reset_index(drop=True)
red2

## ex. A기업 n개의 투자설명서에 1개의 상장일 (1,015개) ▶ 1:1 대응 (526개)

(1059, 8)
(1039, 5)
(1015, 10)


Symbol               object
crp_nm               object
rcp_dt               object
rcp_no               object
rpt_nm               object
redate       datetime64[ns]
name                 object
listdate             object
date                 object
list_date    datetime64[ns]
dtype: object

In [74]:
red2.to_csv("D:\\cg\\data\\red2.csv", encoding="ms949", header=True,index=False)